# Intro
In this module we will create a remote MCP server that represents an employee management system.
1. You will first run the server
2. You will serve it with a global URL using ngrok
3. You will run queries to the server using Maestro

## Start the Employees MCP server

In [ ]:
import subprocess
import time

# Global variable to track the server process
server_process = None

def start_mcp_server():
    """Start the MCP server as a subprocess"""
    global server_process
    
    if server_process and server_process.poll() is None:
        print("⚠️  Server already running")
        return
    
    server_process = subprocess.Popen(
        ["python", "mcp_server.py"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    time.sleep(2)  # Give server time to start
    
    # Check if server started successfully
    if server_process.poll() is None:
        print("✓ MCP Server running on http://localhost:8000/mcp")
    else:
        print("❌ Server failed to start")
        stdout, stderr = server_process.communicate()
        print(f"Error: {stderr.decode()}")

def stop_mcp_server():
    """Stop the MCP server"""
    global server_process
    
    if not server_process or server_process.poll() is not None:
        print("⚠️  Server not running")
        return
    
    server_process.terminate()
    try:
        server_process.wait(timeout=5)
        print("✓ MCP Server stopped")
    except subprocess.TimeoutExpired:
        server_process.kill()
        server_process.wait()
        print("✓ MCP Server force stopped")
    
    server_process = None

# Start the server
start_mcp_server()

(Optional) If you want to terminate the MCP server you can use the following utility or terminate the Jupyter kernel

In [ ]:
stop_mcp_server()

## Install ngrok to make the server accessible from the web

1. Create a [free account](https://dashboard.ngrok.com/signup)
2. Install [ngrok](https://ngrok.com/download/mac-os)
3. Follow the authentication instructions by running:
```sh
ngrok config add-authtoken <your token>
```

4. in the command line run:
`ngrok http 8000`
5. You’ll get a publicly accessible URL like: https://abc123.ngrok-free.app

Important: Save this URL, you’ll need it for AI21 Maestro integration

In [ ]:
# Now let's test the connection
from ai21 import AI21Client
import os

# paste your API key here or set it as an environment variable
API_KEY = os.environ.get("AI21_API_KEY")

client = AI21Client(api_key=API_KEY)

In [ ]:
run = client.beta.maestro.runs.create_and_poll(
    input="What is the salary of employee EMP001?",
    tools=[
        {
            "type": "mcp",
            "server_url": "https://<ngrok-id>.ngrok-free.app/mcp", # change this to your ngrok URL
            "server_label": "Employees",
        },
    ],
    budget="medium",
)

print("id:", run.id)
print("Status:", run.status)
print("Result:", run.result)

## Next steps:
* Review the run in the traces page
* Run more questions like:
  * What is the salary of employee EMP001?
  * Show me all employees in the Engineering department
  * Who reports to EMP005?
  * What are the department statistics?
* Use requirements